# Forecasting Evaluation

This notebook shows you how to evaluate the pre-trained checkpoints against the Black Sea and Scalar Flow datasets. It requires that you install the package with the `lightning` extra, i.e. `pip install -e '.[lightning]'`.

In [1]:
from pathlib import Path

import torch
from pytorch_lightning import Trainer

import finite_element_networks as fen
from finite_element_networks import FENDomainInfo
from finite_element_networks.lightning import BlackSeaDataModule, ScalarFlowDataModule, SequenceRegressionTask
from finite_element_networks.utils import load_checkpoint

In [2]:
dataset_name = "black-sea"
model_name = "tfen"

In [3]:
project_root = Path(fen.__file__).resolve().parent.parent
ckpt_dir = project_root / "checkpoints"
data_dir = project_root / "data"

In [4]:
ckpt = torch.load(ckpt_dir / dataset_name / f"{model_name}.pt")
model = load_checkpoint(ckpt)

In [5]:
if dataset_name == "black-sea":
    dm = BlackSeaDataModule(data_dir / dataset_name, FENDomainInfo.from_domain)
if dataset_name == "scalar-flow":
    dm = ScalarFlowDataModule(data_dir / dataset_name, FENDomainInfo.from_domain)

In [6]:
task = SequenceRegressionTask(model)

In [7]:
gpus = 1 if torch.cuda.is_available() else 0
trainer = Trainer(gpus=gpus)
trainer.test(task, datamodule=dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/ml/.pyenv/versions/3.9.10/envs/fen/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:659: UserWarning: Your `test_dataloader` has `shuffle=True`, it is strongly recommended that you turn this off for val/test/predict dataloaders.
  rank_zero_warn(
/home/ml/.pyenv/versions/3.9.10/envs/fen/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test/mae': 0.8493268489837646, 'test/nfe': 65.71186065673828}
--------------------------------------------------------------------------------


[{'test/nfe': 65.71186065673828, 'test/mae': 0.8493268489837646}]